In [2]:
import youtube_dl
import re
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

In [3]:
WAV_DIR = 'wav_files/'
genre_dict = {
            '/m/064t9': 'Pop_music',
            '/m/0glt670': 'Hip_hop_music',
            '/m/0y4f8': 'Vocal',
            '/m/06cqb': 'Reggae',

            }

genre_set = set(genre_dict.keys())

genre_labels=list(genre_dict.values())


In [4]:
temp_str = []
with open('data-files/csv_files/unbalanced_train_segments.csv', 'r') as f:
    temp_str = f.readlines()

In [5]:
data = np.ones(shape=(1,4)) 
for line in tqdm(temp_str):
    line = re.sub('\s?"', '', line.strip())
    elements = line.split(',')
    common_elements = list(genre_set.intersection(elements[3:]))
    if  common_elements != []:
        data = np.vstack([data, np.array(elements[:3]
                                         + [genre_dict[common_elements[0]]]).reshape(1, 4)])

df = pd.DataFrame(data[1:], columns=['url', 'start_time', 'end_time', 'class_label'])

100%|██████████| 2041792/2041792 [00:52<00:00, 38776.80it/s]


In [6]:
df.head()


,url,start_time,end_time,class_label
0,--1rvyPa8UM,40.000,50.000,Hip_hop_music
1,--4kp9W7cNY,160.000,170.000,Reggae
2,--Fti-jdXEI,240.000,250.000,Vocal
3,--GXulx19TI,80.000,90.000,Pop_music
4,--MLNYqE2ec,30.000,40.000,Pop_music


In [7]:
df['class_label'].value_counts()


Pop_music        8659
Hip_hop_music    7505
Vocal            3468
Reggae           2813
Name: class_label, dtype: int64

In [6]:
with open('your_file.txt', 'r') as f:
    for item in indice:
        f.read("%s\n" % item)

Pop_music        8631
Hip_hop_music    7199
Vocal            3508
Reggae           3107
Name: class_label, dtype: int64

In [7]:
# take only 1k  audio clips - to make the data more balanced
np.random.seed(10)
drop_values=list(df['class_label'].value_counts())
x=[1000,1000,1000,1000]
drop_values=[a_i - b_i for a_i, b_i in zip(drop_values, x)]

for value,label in zip(drop_values,genre_labels) :
    drop_indices = np.random.choice(df[df['class_label'] == label].index, size=value, replace=False)
    df.drop(labels=drop_indices, axis=0, inplace=True)
    df.reset_index(drop=True, inplace=False)

# Time to INT 
df['start_time'] = df['start_time'].map(lambda x: np.int32(np.float(x)))
df['end_time'] = df['end_time'].map(lambda x: np.int32(np.float(x)))

In [8]:
df['class_label'].value_counts()

Hip_hop_music    1000
Pop_music        1000
Vocal            1000
Reggae           1000
Name: class_label, dtype: int64

Example:<br>
Step 1:<br>
`ffmpeg -ss 5 -i $(youtube-dl -f 140 --get-url 'https://www.youtube.com/embed/---1_cCGK4M') -t 10 -c:v copy -c:a copy test.mp4`<br>
Starting time is 5 seconds, duration is 10s.

Refer: https://github.com/rg3/youtube-dl/issues/622

Step 2:<br>
`ffmpeg -i test.mp4 -vn -acodec pcm_s16le -ar 44100 -ac 1 output.wav` <br>
PCM-16, 44k sampling, 1-channel (Mono)
<br>
Refer: https://superuser.com/questions/609740/extracting-wav-from-mp4-while-preserving-the-highest-possible-quality

In [ ]:
for i, row in tqdm(df.iterrows()):
    url = "'https://www.youtube.com/embed/" + row['url'] + "'"
    file_name = str(i)+"_"+row['class_label']
    
    try:
        command_1 = "ffmpeg -ss " + str(row['start_time']) + " -i $(youtube-dl -f 140 --get-url " +\
                    url + ") -t 15 -c:v copy -c:a copy " + file_name + ".mp4"

        command_2 = "ffmpeg -i "+ file_name +".mp4 -vn -acodec pcm_s16le -ar 44100 -ac 1 " + WAV_DIR + file_name + ".wav"

        command_3 = 'rm ' + file_name + '.mp4' 

        # Run the 3 commands
        os.system(command_1 + ';' + command_2 + ';' + command_3 + ';')
    
    except:
        print(i, url)
        pass

4000it [00:00, 13385.93it/s]


4000